# Recursion Schemes

In [2]:
:ext DeriveFunctor

import Control.Arrow
import Data.Monoid

newtype Fix f = Fix { unfix :: f (Fix f) }

data List a b = End | Cons a b deriving (Show, Functor)

## Catamorphism
Generic functor crusher

In [4]:
type Algebra f a = f a -> a

cata :: Functor f => Algebra f a -> Fix f -> a
cata fn = unfix >>> fmap (cata fn) >>> fn

exampleAlg :: Monoid m => Algebra (List m) m
exampleAlg End = mempty
exampleAlg (Cons a b) = a <> b

let exList = Fix $ Cons 4 (Fix $ Cons 3 (Fix $ Cons 2 $ Fix End))
cata exampleAlg exList :: Product Int
cata exampleAlg exList :: Sum Int

Product {getProduct = 24}

Sum {getSum = 9}

## Anamorphism
The dual of a catamorphism: a generic un-folder.

In [4]:
type Coalgebra f a = a -> f a

ana :: Functor f => Coalgebra f a -> a -> Fix f
ana fn = Fix <<< fmap (ana fn) <<< fn

exampleCoalg :: Coalgebra (List (Sum Int)) (Sum Int)
exampleCoalg 0 = End
exampleCoalg v = Cons v $ v - 1

cata exampleAlg $ ana exampleCoalg 5 :: Sum Int

Sum {getSum = 15}

## Paramorphism
`cata` destroys info about the data structure that we are transforming. Using Paramorphisms, we retain all info about the original structure.

In [5]:
type RAlgebra f a = f (Fix f, a) -> a

para :: Functor f => RAlgebra f a -> Fix f -> a
para fn = unfix >>> fmap (id &&& para fn) >>> fn

exampleRAlg :: RAlgebra (List Int) Int
exampleRAlg End = 0
exampleRAlg (Cons a (Fix (Cons 3 _), b)) = a + 3 -- start over if previous value was 3
exampleRAlg (Cons a (_, b)) = a + b

para exampleRAlg exList :: Int
cata exampleAlg exList :: Sum Int

7

Sum {getSum = 9}

## Apomorphism
The dual of `para`

In [6]:
type RCoalgebra f a = a -> f (Either (Fix f) a)

apo :: Functor f => RCoalgebra f a -> a -> Fix f
apo fn = Fix <<< fmap (id ||| apo fn) <<< fn

exampleRCoalg :: (Num a, Eq a, Ord a) => RCoalgebra (List a) a
exampleRCoalg 5 = Cons 5 (Left $ Fix $ Cons 555 $ Fix End) -- manually create the rest of the list
exampleRCoalg n
  | n > 10 =  End
  | otherwise = Cons n $ Right (n + 2)
  
cata exampleAlg $ apo exampleRCoalg 2 :: Sum Int
cata exampleAlg $ apo exampleRCoalg 1 :: Sum Int

Sum {getSum = 30}

Sum {getSum = 564}

## Histomorphism
A catamorphism that remembers all previous results

In [30]:
:ext ScopedTypeVariables

data Attr f a = Attr  -- like the cofree comonad
              { attribute :: a
              , hole      :: f (Attr f a)
              }

type CVAlgebra f a = f (Attr f a) -> a

--hist :: forall a f. Functor f => CVAlgebra f a -> Fix f -> a
hist fn = unfix >>> fmap worker >>> fn
  where
    --worker :: Fix f -> Attr f a
    worker = unfix >>> fmap worker >>> fn &&& id >>> uncurry Attr
    
data Nat a = Next a | Zero deriving Functor
    
-- fibonacci sequence
exampleCVAlgebra :: CVAlgebra Nat Int
exampleCVAlgebra (Zero) = 0
exampleCVAlgebra (Next (Attr _ Zero)) = 1
exampleCVAlgebra (Next (Attr a (Next (Attr b _)))) = a + b

let exNat = foldr (.) id (replicate 13 $ Fix . Next) $ Fix Zero

hist exampleCVAlgebra exNat

sineHisto :: Double -> Double -> Double -> Int -> Double
sineHisto freq sr phase samp = hist alg numSamp
  where
    numSamp = foldr (.) id (replicate samp $ Fix . Next) $ Fix Zero
    b = 2 * pi * freq / sr
    twoCos = 2 * cos b
    alg :: CVAlgebra Nat Double
    alg Zero = sin $ phase * b
    alg (Next (Attr _ Zero)) = sin $ (phase + 1) * b
    alg (Next (Attr a (Next (Attr b _)))) = twoCos * a - b
   
sineHisto 1 (2 * pi) 0 666
sin 666

233

-1.7641645813307405e-2

-1.764164581327013e-2

## Futumorphism
The dual of `histo`

In [8]:
data CoAttr f a -- like Free monad
     = Pure a
     | Free (f (CoAttr f a))
     
type CVCoalgebra f a = a -> f (CoAttr f a)

futu :: Functor f => CVCoalgebra f a -> a -> Fix f
futu fn = Fix <<< fmap worker <<< fn
  where
    worker (Pure a) = futu fn a
    worker (Free c) = Fix $ fmap worker c

## Using the Recursion Schemes Library
How to use recursion schemes with simple types such as [a], instead of our `List` type.
This involves using the `Base` functor

In [9]:
:ext TypeFamilies

type family Base t :: * -> *

Type families are "functions on the type level". For example:

In [10]:
type family Something t
type Foo = Int
type Bar = Float
type instance Something Foo = Bar
:t (4 :: Something Foo)

(4 :: Something Foo) :: Bar

In the above, wherever we have `Something Foo` then it will be resolved to `Bar`. We have to declare a type instance for Foo like declaring an instance of a typeclass. 
This provides a powerful facility with which to associate one type with another.

Returning to the definition of `Base`, we see that passing in a concrete type results in a type that is parameterized with one additional variable. This is the same kind as our `List` implementation.

In [11]:
type instance Base [a] = List a

Now we can use Base [a] to refer to `List a`

In [12]:
:ext FlexibleContexts

class (Functor (Base t)) => Recursive t where
  project :: t -> Base t t -- takes a type and transforms it into it's Base form
  cata' :: (Base t a -> a) -> t -> a
  cata' fn = c where c = fn . fmap c . project
  {-# MINIMAL project #-}

This frees us from having to work with `Fix`. We can instead use regular types.

In [13]:
instance Recursive [a] where
  project (x:xs) = Cons x xs
  project [] = End

sumList :: Num a => [a] -> a
sumList = cata' go
  where
    go End = 0
    go (Cons a acc) = a + acc
    
sumList [1,2,3,4,5,6]

21

Another useful instance is for `Natural`

In [14]:
import Numeric.Natural

type instance Base Natural = Maybe

instance Recursive Natural where
  project 0 = Nothing
  project n = Just (n-1)

The `recursion-schemes` package has a template haskell splice to generate all this boiler plate for us.
Given a type like this:

In [15]:
data Tree a
   = Node (Tree a) a (Tree a)
   | Leaf a

we can just do this:

In [16]:
--import Data.Functor.Foldable.TH

--makeBaseFunctor ''Tree

## Corecursive Class
reverse the arrows of `Recursive`

In [17]:
class Functor (Base t) => Corecursive t where
  embed :: Base t t -> t
  ana :: (a -> Base t a) -> a -> t
  ana fn = c where c = embed . fmap c . fn
  {-# MINIMAL embed #-}
  
instance Corecursive [a] where
  embed (Cons x xs) = x:xs
  embed End = []

`makeBaseFunctor` will also create `Corecursive` instances.